In [2]:
using TwoSD

using Distributions, JuMP, CPLEX
using LinearAlgebra

In [3]:
#import Pkg; 
#Pkg.add("CSV")
#Pkg.add("DataFrames")

In [4]:
L = [0.7, 0];
U = [296.4, 49.6];
b0 = 3.157054;
b1 = 0.044285;
b2 = 0.194257;


In [5]:
model = direct_model(optimizer_with_attributes(
    CPLEX.Optimizer, CPLEX.PassNames() => true
))

@variables(model, begin
    x1 >= 0
    x2 >= 0
    end);

@variables(model, begin
    ya >= 0
    yb >= 0
    end);

In [11]:
@objective(model, Min, 0.1*x1 + 0.5*x2 - 3*ya - 5*yb);
#stage 1
@constraint(model, c4b, x1+x2 <= 200);
@constraint(model, c4c, x1-0.5*x2 >= 0);
@constraint(model, c7, x1 >= L[1]);
@constraint(model, c8, x2 >= L[2]);
@constraint(model, c9, x1 <= U[1]);
@constraint(model, c10, x2 <= U[2]);
#stage 2
@constraint(model, c4d, ya <= 8);
@constraint(model, c4e, 2*yb <= 24);
@constraint(model, c4f, 3*ya+2*yb <= 36);
@constraint(model, c4g, -b1*x1-b2*x2+ya+yb <= b0);


LoadError: An object of name c4b is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, e.g.,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :c4b)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:c4b]`.


In [12]:
using Random
rng = Random.MersenneTwister(1234)
ei=[1.4101, -2.362, -3.5352, 0.611, -0.3618, 0.8542, -1.9103, 1.389, 0.0728, -1.8295, 0.7488, 0.4133, 1.3236, -0.7694, 1.0886, 1.0962, 0.277, -0.2098, -2.161, -1.2303, -1.0503, 1.3364, -3.4795, -0.1771, -1.134, -0.5395, 1.1083, -0.2256, 0.3631, 1.8471, -1.4043, 1.8329, -4.0272, 1.9148, -1.3614, -0.1524, 0.9225, 0.1433, -0.3837, -0.84, -1.0514, -0.761, -0.3821, 1.5474, 1.3573, -1.4879, 1.3075, -1.2074, 1.2319, 1.7592, 0.9818, -0.1853, 2.1384, -3.439, 0.2816, 1.6725, 0.1815, 2.1852, 1.1765, -1.0654, 0.5449, 0.723, 1.2807, 0.1692, 1.2573, -0.1124, 1.0141, 0.3815, 1.6026, -1.9544, 1.0052, -0.3862, -3.6945, 2.2143, 0.1703, -3.9045, 1.2101, 0.0729, -2.2731, 0.8649, -1.2306, 0.735, -0.0872, 0.1219, 0.0538, 0.879, -0.605, 1.1436, 2.585, 0.0945, 0.8415, 0.8671, 0.3727, -0.8877, 0.0753, 1.1965, -0.0449, -2.1413]


98-element Vector{Float64}:
  1.4101
 -2.362
 -3.5352
  0.611
 -0.3618
  0.8542
 -1.9103
  1.389
  0.0728
 -1.8295
  0.7488
  0.4133
  1.3236
  ⋮
 -0.605
  1.1436
  2.585
  0.0945
  0.8415
  0.8671
  0.3727
 -0.8877
  0.0753
  1.1965
 -0.0449
 -2.1413

In [13]:
# A function that generates a random scenario.
# A realization is a Vector of pairs: Position => value
# Position is constructed by a constraint(row) and a variable(column)
# If the randomness is on the RHS, replace it with "RHS"
function mystoc()
    d = rand(rng, ei) #Create a function that returns a random sample of εti each time it is called
    binding = [Position(c4g, "RHS") => d+b0]
    return OneRealization(binding)
end;


In [14]:
user_mean = copy([b0]);

In [15]:
split_position = Position(c4d, ya);


In [16]:
solution = solve_sd(model, split_position, user_mean, mystoc)

=== dumpCore ===
lp = 0x7a7e360
type = 0
objsen = 1
rows	rname	senx	rhsx	mar=10
0	c4b	L	200
1	c4c	G	0
2	c7	G	0.7
3	c8	G	0
4	c9	L	296.4
5	c10	L	49.6
6	c4d	L	8
7	c4e	L	24
8	c4f	L	36
9	c4g	L	3.15705
cols	cname	ctype	bdl	bdu	mac=4
0	x1	C	0	1e+20
1	x2	C	0	1e+20
2	ya	C	0	1e+20
3	yb	C	0	1e+20
=== dumpTime ===
nr = 2
nc = 2
stgNames = TIME1 TIME2
row = 0 6
col = 0 2
=== dumpStoc ===
type = EXT_GENERATOR
sim = 1
numOmega = 1
numGroups = 1
groupBeg = 0
row	col	mean = ..
9	-1	3.157054

Optimal objective function value for (relaxed) mean value problem = -41.368826
Lower bounds computed = -81.209	
Starting two-stage stochastic decomposition.
Stage optimization problem for given input s_t = (x_t, omega_t) is in the following form : 

				 h_t(s_t) = c_t*x_t + min d_t*u_t
				                      s.t. D_t u_t = b_t - C_t x_t,

with the following linear dynamics: x_{t+} = a_{t+} + A_{t+}x_t + B_{t+}u_t.

-------------------------------------------------------------------------------------------------

TwoSD.SDResult(Dict{ConstraintRef, Int64}(c7 : x1 ≥ 0.7 => 3, c4g : -0.044285 x1 - 0.194257 x2 + ya + yb ≤ 3.157054 => 10, c4b : x1 + x2 ≤ 200.0 => 1, c4e : 2 yb ≤ 24.0 => 8, c10 : x2 ≤ 49.6 => 6, c4d : ya ≤ 8.0 => 7, c9 : x1 ≤ 296.4 => 5, c4c : x1 - 0.5 x2 ≥ 0.0 => 2, c8 : x2 ≥ 0.0 => 4, c4f : 3 ya + 2 yb ≤ 36.0 => 9…), Dict{VariableRef, Int64}(yb => 4, x2 => 2, x1 => 1, ya => 3), Dict{Int64, ConstraintRef{Model, C, ScalarShape} where C}(5 => c9 : x1 ≤ 296.4, 4 => c8 : x2 ≥ 0.0, 6 => c10 : x2 ≤ 49.6, 7 => c4d : ya ≤ 8.0, 2 => c4c : x1 - 0.5 x2 ≥ 0.0, 10 => c4g : -0.044285 x1 - 0.194257 x2 + ya + yb ≤ 3.157054, 9 => c4f : 3 ya + 2 yb ≤ 36.0, 8 => c4e : 2 yb ≤ 24.0, 3 => c7 : x1 ≥ 0.7, 1 => c4b : x1 + x2 ≤ 200.0…), Dict{Int64, ConstraintRef{Model, C, ScalarShape} where C}(5 => c9 : x1 ≤ 296.4, 4 => c8 : x2 ≥ 0.0, 6 => c10 : x2 ≤ 49.6, 7 => c4d : ya ≤ 8.0, 2 => c4c : x1 - 0.5 x2 ≥ 0.0, 10 => c4g : -0.044285 x1 - 0.194257 x2 + ya + yb ≤ 3.157054, 9 => c4f : 3 ya + 2 yb ≤ 36.0, 8 => c4e : 

In [20]:
x1_opt = decision.(x1, Ref(solution), CompromiseSolution)


182.292798

In [21]:
x2_opt = decision.(x2, Ref(solution), CompromiseSolution)


17.707202

In [22]:
w = [15.134981398344, 11.941931798344, 14.550212598344, 15.000882398344, 15.592158198344, 15.468632498344, 16.150943398344, 16.154748198344, 14.776682198344, 13.320124098344, 15.227133998344, 14.840152198344, 14.128754298344, 13.558269898344, 13.987785998344, 14.770237398344, 17.373804898344, 14.677876698344, 14.145343498344, 15.549026398344, 13.464649598344, 12.726527698344, 14.539808798344, 14.774675898344, 15.958699898344, 10.210564198344, 16.760927498344, 16.938005998344, 16.242114498344, 11.904857198344, 11.556800098344, 14.871131998344, 13.180147798344, 11.843539998344, 12.302865498344, 14.577752698344, 14.177073198344, 15.496405698344, 15.859696498344, 15.257882198344, 15.398409598344, 16.173108598344, 15.777363898344, 15.230310698344, 12.661679898344, 16.623879498344, 12.901197398344, 14.621214398344, 12.481612698344, 16.122340698344, 14.835680398344, 15.214561098344, 14.697038798344, 12.277634798344, 14.204043498344, 14.726157398344, 11.126366698344, 15.005921198344, 14.757370298344, 14.062959398344, 14.553238798344, 15.123329398344, 15.366804598344, 12.367278198344, 11.415819798344, 13.544310098344, 14.489130298344, 11.863234798344, 15.444953298344, 14.667730698344, 14.340032798344, 14.375765798344, 12.503126698344, 16.750900698344, 14.845629098344, 14.160072898344, 10.612133898344, 14.836418498344, 14.572485298344, 12.987324198344, 16.616502598344, 16.623167498344, 13.735377398344, 16.273168798344, 15.226887298344, 14.774545098344, 12.046902198344, 15.033945298344, 12.579364298344, 15.971091398344, 15.854428798344, 15.567052498344, 15.267570798344, 16.702146598344, 16.089078198344, 14.667549398344, 16.294564498344, 12.963425798344]


98-element Vector{Float64}:
 15.134981398344
 11.941931798344
 14.550212598344
 15.000882398344
 15.592158198344
 15.468632498344
 16.150943398344
 16.154748198344
 14.776682198344
 13.320124098344
 15.227133998344
 14.840152198344
 14.128754298344
  ⋮
 12.046902198344
 15.033945298344
 12.579364298344
 15.971091398344
 15.854428798344
 15.567052498344
 15.267570798344
 16.702146598344
 16.089078198344
 14.667549398344
 16.294564498344
 12.963425798344

In [23]:
function second_stage(w)
    model_2 = Model(CPLEX.Optimizer)

    @variable(model_2, ya >= 0);
    @variable(model_2, yb >= 0);
    @objective(model_2, Max, 3*ya+5*yb);
    @constraint(model_2, c21, ya <= 8);
    @constraint(model_2, c22, 2*yb <= 24);
    @constraint(model_2, c23, 3*ya+2*yb <= 36);
    @constraint(model_2, c24, ya+yb <= w);
    optimize!(model_2)
    @assert(termination_status(model_2) == OPTIMAL)
#     @show objective_value(model_2)
    obj = objective_value(model_2)
    return obj
end


avg_obj::Float64 = 0.0
all_obj = []

for wi in w
    obj = second_stage(wi)
    push!(all_obj, obj);
end

# print(all_obj)
    

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
Tried aggregator 1 time.
LP Presolve eliminated 2 rows and 0 columns.
Reduced LP has 2 rows, 2 columns, and 4 nonzeros.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =            72.000000
Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
Tried aggregator 1 time.
LP Presolve eliminated 2 rows and 0 columns.
Reduced LP has 2 rows, 2 columns, and 4 nonzeros.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =            72.000000
Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
Tried aggregator 1 time.
LP Presolve eliminated 2 rows and 0 columns.
Reduced LP has 2 rows, 2 columns, and 4 nonzeros.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =            72.000000
Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
Tried aggregator 1 time.
LP Presolve eliminated 2 rows and 0 

In [24]:
print(all_obj)

Any[69.404944195032, 59.70965899172, 67.650637795032, 69.002647195032, 70.776474595032, 70.40589749503201, 72.0, 72.0, 68.330046595032, 63.960372295032, 69.68140199503199, 68.520456595032, 66.386262895032, 64.674809695032, 65.963357995032, 68.310712195032, 72.0, 68.03363009503201, 66.436030495032, 70.647079195032, 64.393948795032, 62.179583095032, 67.61942639503201, 68.324027695032, 71.876099695032, 51.052820991720004, 72.0, 72.0, 72.0, 59.52428599172001, 57.78400049172, 68.613395995032, 63.540443395032, 59.217699991719996, 60.908596495032, 67.733258095032, 66.531219595032, 70.489217095032, 71.57908949503201, 69.773646595032, 70.195228795032, 72.0, 71.332091695032, 69.690932095032, 61.985039695032, 72.0, 62.703592195032, 67.863643195032, 61.444838095032, 72.0, 68.507041195032, 69.64368329503199, 68.091116395032, 60.832904395032, 66.612130495032, 68.178472195032, 55.63183349172, 69.017763595032, 68.272110895032, 66.188878195032, 67.659716395032, 69.369988195032, 70.100413795032, 61.1018